<a href="https://colab.research.google.com/github/ADMoreau/EEG_CPC/blob/master/eegcpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install braindecode
!pip install mne
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np
import pandas as pd
import mne
import logging
import os.path
import time
from collections import OrderedDict
import sys
import matplotlib.pyplot as plt

from braindecode.models.deep4 import Deep4Net
from braindecode.datasets.bcic_iv_2a import BCICompetition4Set2A
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import (
    LossMonitor,
    MisclassMonitor,
    RuntimeMonitor,
)
from braindecode.experiments.stopcriteria import MaxEpochs, NoDecrease, Or
from braindecode.datautil.iterators import BalancedBatchSizeIterator
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.datautil.splitters import split_into_two_sets
from braindecode.torch_ext.constraints import MaxNormDefaultConstraint
from braindecode.torch_ext.util import set_random_seeds, np_to_var
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import (
    highpass_cnt,
    bandpass_cnt,
    exponential_running_standardize,
)
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne


     |████████████████████████████████| 327kB 20.9MB/s 
     |████████████████████████████████| 6.4MB 55.9MB/s 
  Created wheel for braindecode: filename=Braindecode-0.4.85-cp36-none-any.whl size=71078 sha256=69a0eaf62ae4d495b249f1316b0a932c61bc66e4216a31f4bbe5165ceccf9458
  Stored in directory: /root/.cache/pip/wheels/72/07/db/c243929cde1ab13cdb3da732b19244e93ed2c44a30894b8c4a
Successfully built braindecode


In [0]:
from braindecode.datasets.bbci import  BBCIDataset
test_cnt = BBCIDataset(filename="/content/drive/My Drive/eeg/high-gamma-dataset/data/test/1.mat").load()
train_cnt = BBCIDataset(filename="/content/drive/My Drive/eeg/high-gamma-dataset/data/train/1.mat").load()

Creating RawArray with float64 data, n_channels=128, n_times=616535
    Range : 0 ... 616534 =      0.000 ...  1233.068 secs
Ready.
Creating RawArray with float64 data, n_channels=128, n_times=1225545
    Range : 0 ... 1225544 =      0.000 ...  2451.088 secs
Ready.


In [0]:
marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                              ('Rest', [3]), ('Feet', [4])])
clean_ival = [0, 4000]
train_set_for_cleaning = create_signal_target_from_raw_mne(train_cnt, marker_def,
                                                  clean_ival)
train_clean_trial_mask = np.max(np.abs(train_set_for_cleaning.X), axis=(1, 2)) < 800
test_set_for_cleaning = create_signal_target_from_raw_mne(test_cnt, marker_def,
                                                  clean_ival)
test_clean_trial_mask = np.max(np.abs(test_set_for_cleaning.X), axis=(1, 2)) < 800

In [0]:
print("Clean trials: {:3d}  of {:3d} ({:5.1f}%)".format(
        np.sum(train_clean_trial_mask),
        len(train_set_for_cleaning.X),
        np.mean(train_clean_trial_mask) * 100))
print("Clean trials: {:3d}  of {:3d} ({:5.1f}%)".format(
        np.sum(test_clean_trial_mask),
        len(test_set_for_cleaning.X),
        np.mean(test_clean_trial_mask) * 100))

Clean trials: 319  of 320 ( 99.7%)
Clean trials: 160  of 160 (100.0%)


In [0]:
# now pick only sensors with C in their name
# as they cover motor cortex
C_sensors = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'C4', 'CP5',
              'CP1', 'CP2', 'CP6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2',
              'C6',
              'CP3', 'CPz', 'CP4', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h',
              'FCC5h',
              'FCC3h', 'FCC4h', 'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 'CCP6h',
              'CPP5h',
              'CPP3h', 'CPP4h', 'CPP6h', 'FFC1h', 'FFC2h', 'FCC1h', 'FCC2h',
              'CCP1h',
              'CCP2h', 'CPP1h', 'CPP2h']
train_cnt = train_cnt.pick_channels(C_sensors)
test_cnt = test_cnt.pick_channels(C_sensors)

In [0]:
low_cut_hz = 4
train_cnt = resample_cnt(train_cnt, 250.0)
train_cnt = mne_apply(
    lambda a: highpass_cnt(
        a, low_cut_hz, train_cnt.info['sfreq'], filt_order=3, axis=1),
    train_cnt)

test_cnt = resample_cnt(test_cnt, 250.0)
test_cnt = mne_apply(
    lambda a: highpass_cnt(
        a, low_cut_hz, test_cnt.info['sfreq'], filt_order=3, axis=1),
    test_cnt)

This is not causal, uses future data....


Creating RawArray with float64 data, n_channels=44, n_times=612772
    Range : 0 ... 612771 =      0.000 ...  2451.084 secs
Ready.


This is not causal, uses future data....


Creating RawArray with float64 data, n_channels=44, n_times=308267
    Range : 0 ... 308266 =      0.000 ...  1233.064 secs
Ready.


In [0]:
# Trial interval, start at -500 already, since improved decoding for networks
ival = [-500, 4000]

train_dataset = create_signal_target_from_raw_mne(train_cnt, marker_def, ival)
train_dataset.X = train_dataset.X[train_clean_trial_mask]
train_dataset.y = train_dataset.y[train_clean_trial_mask]

test_dataset = create_signal_target_from_raw_mne(test_cnt, marker_def, ival)
test_dataset.X = test_dataset.X[test_clean_trial_mask]
test_dataset.y = test_dataset.y[test_clean_trial_mask]

In [0]:
valid_dataset_fraction = 0.8
train_dataset, valid_dataset = split_into_two_sets(train_dataset,
                                            valid_dataset_fraction)

In [0]:
train_dataset.X.shape, train_dataset.y.shape, valid_dataset.X.shape, valid_dataset.y.shape, test_dataset.X.shape, test_dataset.y.shape

((255, 44, 1125), (255,), (64, 44, 1125), (64,), (160, 44, 1125), (160,))

In [0]:
import torch.nn as nn
import torch

In [0]:
class Expression(torch.nn.Module):
    """
    Compute given expression on forward pass.
    Parameters
    ----------
    expression_fn: function
        Should accept variable number of objects of type
        `torch.autograd.Variable` to compute its output.
    """

    def __init__(self, expression_fn):
        super(Expression, self).__init__()
        self.expression_fn = expression_fn

    def forward(self, *x):
        return self.expression_fn(*x)

    def __repr__(self):
        if hasattr(self.expression_fn, "func") and hasattr(
            self.expression_fn, "kwargs"
        ):
            expression_str = "{:s} {:s}".format(
                self.expression_fn.func.__name__, str(self.expression_fn.kwargs)
            )
        elif hasattr(self.expression_fn, "__name__"):
            expression_str = self.expression_fn.__name__
        else:
            expression_str = repr(self.expression_fn)
        return (
            self.__class__.__name__
            + "("
            + "expression="
            + str(expression_str)
            + ")"
        )

def identity(x):
  return x

def dot_norm_exp(a,b):
  dot = torch.sum(a * b, dim=1)
  aa = torch.sum((a**2),dim=1)**0.5
  bb = torch.sum((b**2),dim=1)**0.5
  dot_norm = dot/(aa*bb)
  ret = torch.exp(dot_norm)
  return ret

def dot_norm(a,b):
  dot = torch.sum(a * b, dim=1)
  aa = torch.sum((a**2),dim=1)**0.5
  bb = torch.sum((b**2),dim=1)**0.5
  dot_norm = dot/(aa*bb)
  return dot_norm

def dot(a,b):
  dot = torch.sum(a * b, dim=1)
  return dot

def norm_euclidean(a,b):
  aa = (torch.sum((a**2),dim=1)**0.5).unsqueeze(dim=1)
  bb = (torch.sum((b**2),dim=1)**0.5).unsqueeze(dim=1)
  return (torch.sum(((a/aa-b/bb)**2),dim=1)**0.5)

class channel_encoder(nn.Module):
  def __init__(self):
    super(channel_encoder, self).__init__()
    self.channel_encoder = nn.Sequential(
        nn.Conv1d(in_channels = 1,
                  out_channels = 25,
                  kernel_size = 10,
                  stride=1,
                  bias=False),
        nn.BatchNorm1d(25, momentum=.1, affine=True),
        nn.ELU(),
        nn.MaxPool1d(kernel_size=3, stride=3),
        Expression(identity),
        nn.Dropout(p=.5)
    )
  def forward(self, signal):
    return self.channel_encoder(signal)

class encoder(nn.Module):
  def __init__(self, channels):
    super(encoder, self).__init__()
    self.encoders = nn.ModuleList([channel_encoder() for i in range(channels)])
    #LSTMs to focus on all the channels at a single time step
    self.channels_lstms = nn.ModuleList([nn.LSTM(25, 25, num_layers=1, bidirectional=False, batch_first=True) for i in range(channels)])
    #LSTMs to foucs on all previous samples for a single channel
    self.samples_lstms = nn.ModuleList([nn.LSTM(25, 25, num_layers=1, bidirectional=False, batch_first=True) for i in range(channels)])    
    self.prediction_weights = nn.ModuleList([nn.ModuleList([nn.Linear(in_features=1800,
                                                                      out_features=25) 
                                                            for i in range(13)])
                                                            for j in range(channels)])
            
  def forward(self, input_data):
    #input shape = [Batch_size, 44(height), 1125 (width)]
    batch_size = input_data.shape[0]
    channels = input_data.shape[1]
    ''' find the number of windows
    windows = 0
    # window size = 225, overlap = 75
    while (75*i) + 225 <= 1125:
      windows += 1
    '''
    windows = 13 
    
    # get the latents (z) using the encoders, 1 encoder per channel
    z_storage = torch.empty((batch_size, channels, windows, 72, 25)).float().cuda()
    for channel in range(channels):
      for window in range(windows): # 13 is the number of overlapping windows 
        w = 75*window
        encoder_input = input_data[:, channel, w : w + 225]
        encoder_input = encoder_input.unsqueeze(dim=1)
        z = self.encoders[channel](encoder_input)
        z = z.transpose(1,2)
        z_storage[:, channel, window, :, :] = z

    #linear modality
    ######################
    z_targets = torch.empty((batch_size, channels, windows, 25)).float().cuda()
    for channel in range(channels):
      for window in range(windows): 
        z_pred_in = z_storage[:, channel, window, : , :]
        z_pred_in = nn.Flatten(start_dim=1)(z_pred_in)
        z_targets[:, channel, window, :] = self.prediction_weights[channel][window].forward(z_pred_in)
    ######################
    
    #Autoregressive modality
    ##############################
    z_sample_storage = torch.empty((batch_size, channels, windows, 25)).float().cuda()
    for channel in range(channels):
      for window in range(windows):
        # Get the encoder outputs from single channel previous all windows
        z_prev_samples = z_targets[:, channel, :window+1, :]
        # flatten for lstm
        #z_prev_samples = nn.Flatten(start_dim=1, end_dim=3)(z_prev_samples)
        z_samples_prev, _ = self.channels_lstms[channel](z_prev_samples)
        # get final output sample
        z_sample = z_samples_prev[:, -1, :]
        z_sample_storage[:, channel, window, :] = z_sample

    z_preds = torch.empty((batch_size, channels, windows, 25)).float().cuda()
    for channel in range(channels):
      for window in range(windows):   
        # Get the encoder ouputs from previous time window across all channels 
        z_prev_channels = z_sample_storage[:, :, window, :]
        #flatten for lstm
        #z_prev_channels = nn.Flatten(start_dim=1, end_dim=3)(z_prev_channels)
        z_channel_prev, _ = self.samples_lstms[channel](z_prev_channels)
        z_sample = z_channel_prev[:, -1, :]
        z_preds[:, channel, window, :] = z_sample
    #############################

    # Calculate error
    #############################
    losses = []
    for batch in range(batch_size):
      for channel in range(channels):
        for window in range(0, windows-2): # skip reading 0/ start at 0
          c_i_j = z_preds[batch, channel, window, :].unsqueeze(dim=0)
          #for pred_window in range(window+1, windows):
          #z_hat = z_targets[batch, channel, pred_window, :].unsqueeze(dim=0)
          z_hat = z_targets[batch, channel, window + 2, :].unsqueeze(dim=0)
          dot_norm_val = dot_norm_exp(c_i_j, z_hat)
          euc_loss_val = norm_euclidean(c_i_j, z_hat)
          good_term_dot = dot(c_i_j, z_hat)
          dot_terms = [torch.unsqueeze(good_term_dot,dim=0)]

          #for rand in range(8):
          if batch == 0:
            a = np.random.randint(batch_size, size=1)[0]
            b = np.random.randint(channels, size=1)[0]
            c = np.random.randint(windows, size=1)[0]
            bad_term_dot = dot(c_i_j, z_targets[a, b, c, :])
            dot_terms.append(torch.unsqueeze(bad_term_dot, dim=0))

          log_softmax = torch.log_softmax(torch.cat(dot_terms, dim=0), dim=0)
          losses.append(-log_softmax[0,])

    #############################
    

    loss = torch.mean(torch.cat(losses))
    loss.backward()

    return loss


In [0]:
dataset = torch.utils.data.TensorDataset(torch.Tensor(train_dataset.X), torch.Tensor(train_dataset.y))
loader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True)

In [0]:
from tqdm import tqdm
import sys
import time

model = encoder(channels = 44).cuda()
optimizer = torch.optim.Adam(params = model.parameters())
model.train()


for epoch in range(100):
  batch_loss = 0
  now = time.time()
  for l, (X, y) in enumerate(loader):
    loss = model.forward(X.cuda())
    batch_loss += loss.item()
    sys.stdout.write('\rEpoch %d Batch %d Loss %f' % (epoch, l, loss))
    sys.stdout.flush()
  optimizer.step()
  optimizer.zero_grad()
  batch_loss /= l + 1
  then = time.time()
  elapsed = then - now
  print(f"\rEpoch {epoch} Loss {round(batch_loss, 6)} Elapsed Time {round(elapsed, 2)} seconds")
  torch.save(model.state_dict(), os.path.join("/content/drive/My Drive/eeg/encoder_weights.pt"))

Epoch 0 Loss 0.022699 Elapsed Time 174.76 seconds
Epoch 1 Loss 0.013189 Elapsed Time 173.17 seconds
Epoch 2 Loss 0.006269 Elapsed Time 169.48 seconds
Epoch 3 Loss 0.003913 Elapsed Time 166.07 seconds
Epoch 4 Loss 0.002324 Elapsed Time 165.87 seconds
Epoch 5 Loss 0.001676 Elapsed Time 165.04 seconds
Epoch 6 Loss 0.001443 Elapsed Time 165.28 seconds
Epoch 7 Loss 0.001222 Elapsed Time 164.6 seconds
Epoch 8 Loss 0.000933 Elapsed Time 165.88 seconds
Epoch 9 Loss 0.000653 Elapsed Time 167.17 seconds
Epoch 10 Loss 0.000804 Elapsed Time 166.78 seconds
Epoch 11 Loss 0.001034 Elapsed Time 165.87 seconds
Epoch 12 Loss 0.000789 Elapsed Time 165.36 seconds
Epoch 13 Loss 0.000765 Elapsed Time 165.85 seconds
Epoch 14 Loss 0.001077 Elapsed Time 163.51 seconds
Epoch 15 Loss 0.000622 Elapsed Time 164.07 seconds
Epoch 16 Loss 0.001015 Elapsed Time 164.74 seconds
Epoch 17 Loss 0.000704 Elapsed Time 164.6 seconds
Epoch 18 Loss 0.001352 Elapsed Time 162.87 seconds
Epoch 19 Loss 0.000803 Elapsed Time 163.49 

In [0]:
test = torch.Tensor(train_set.X[0, 0, :225]).unsqueeze(dim=0).unsqueeze(dim=0)
print(test.shape)
model = encoder(channels = 44)
out = model.forward(torch.Tensor(train_set.X[0, :, :]).unsqueeze(dim=0))
out

torch.Size([1, 1, 225])


tensor(2.8942, grad_fn=<MeanBackward0>)

In [0]:
rnn = nn.GRU(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)
output.shape

torch.Size([5, 3, 20])

In [0]:
rnn = nn.LSTM(10, 100, 2)
input = torch.randn(5, 300, 10)
h0 = torch.randn(2, 3, 100)
c0 = torch.randn(2, 3, 100)
output, (hn, cn) = rnn(input)
output.shape

torch.Size([5, 300, 100])

In [0]:
import numpy as np
arr = np.ones((1, 1125))
i = 0
while (75*i) + 225 <= 1125:
  i += 1
print(i)

0 225
(1, 225)
75 300
(1, 225)
150 375
(1, 225)
225 450
(1, 225)
300 525
(1, 225)
375 600
(1, 225)
450 675
(1, 225)
525 750
(1, 225)
600 825
(1, 225)
675 900
(1, 225)
750 975
(1, 225)
825 1050
(1, 225)
900 1125
(1, 225)
13
